In [1]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [2]:
%%bash
# 1) ensure latest tokenizers
pip install -q --upgrade transformers tokenizers sentencepiece

# 2) force-clear corrupted cache
rm -rf ~/.cache/huggingface

In [5]:
pip install peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
# Manual LoRA loading approach
# First install: pip install peft bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

PROMPT = (
    "Analisis mendalam putusan ini:\n\n"
    "Putusan Pengadilan Negeri Denpasar\n"
    "Nomor: — /Pid.Sus/2018/PN Dps\n"
    "Terdakwa: Nyoman Suharyeni (kasir)\n"
    "Dakwaan: Pasal 374 ayat 1 KUHP penggelapan dalam jabatan\n"
    "Ringkasan: selama 3 tahun fraud, laporan fiktif, merugikan Rp 2,5 M\n\n"
    "Jawaban:"
)

def load_lora_model_manually():
    try:
        # Step 1: Load base tokenizer
        print("Loading base tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-3-1b-it")

        # Step 2: Load base model
        print("Loading base model...")
        base_model = AutoModelForCausalLM.from_pretrained(
            "unsloth/gemma-3-1b-it",
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )

        # Step 3: Load LoRA adapter
        print("Loading LoRA adapter...")
        from peft import PeftModel
        model = PeftModel.from_pretrained(base_model, "Haeryz/legal-gemma3-id-16bit")

        print("✅ Model loaded successfully!")

        # Step 4: Generate text
        inputs = tokenizer(PROMPT, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=512,
                min_new_tokens=20,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode output
        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = full_text[len(PROMPT):].strip()

        print(f"\n🎯 Generated Legal Analysis:")
        print(f"{generated_text}")

    except ImportError as e:
        print(f"❌ Missing dependency: {e}")
        print("Please install: pip install peft bitsandbytes")

    except Exception as e:
        print(f"❌ Error: {e}")
        print("Try installing bitsandbytes: pip install bitsandbytes")

if __name__ == "__main__":
    print("🔧 Attempting to load your LoRA model...")
    load_lora_model_manually()

🔧 Attempting to load your LoRA model...
Loading base tokenizer...
Loading base model...
Loading LoRA adapter...


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

✅ Model loaded successfully!


W0717 00:50:38.473000 5027 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode



🎯 Generated Legal Analysis:
Berikut adalah analisis mendalam mengenai putusan tersebut, dengan mempertimbangkan berbagai aspek hukum dan sosial:

1.  **Kebenaran Putusan:**
    *   **Penyelidikan dan Bukti:**  Putusan ini didasarkan pada penyelidikan yang dilakukan oleh pihak kepolisian. Penyelidikan ini melibatkan pengumpulan bukti-bukti, termasuk laporan dari terdakwa (Nyoman Suharyeni), saksi, dan dokumen-dokumen terkait.  Bukti-bukti ini harus cukup untuk mendukung tuduhan penggelapan dan tindak pidana lainnya.
    *   **Pengakuan Terdakwa:** Nyoman Suharyeni mengakui melakukan penggelapan dalam jabatan.  Ini merupakan elemen penting dalam proses hukum, menunjukkan bahwa dia menyadari tindakannya telah melanggar aturan dan peraturan perusahaan.
    *   **Kerugian yang Dialami:** Kerugian yang dirugikan mencapai Rp 2,5 juta, yang merupakan bukti kuat terhadap adanya tindak pidana.

2.  **Peran Terdakwa (Nyoman Suharyeni):**
    *   **Pelanggaran Tugas dan Tanggung Jawab:** Sebagai 

In [2]:
# ====== MINI-BENCH 3-MODEL ======
import torch, evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

PROMPT = (
    "Analisis mendalam putusan ini:\n\n"
    "Putusan Pengadilan Negeri Denpasar\n"
    "Nomor: — /Pid.Sus/2018/PN Dps\n"
    "Terdakwa: Nyoman Suharyeni (kasir)  \n"
    "Dakwaan: Pasal 374 ayat 1 KUHP penggelapan dalam jabatan\n"
    "Ringkasan: selama 3 tahun melakukan fraud, laporan fiktif, merugikan Rp 2,5 M\n"
)

MODELS = {
    "🏠 base":       "google/gemma-2b-it",          # same 2 B family
    "⚡ tiny":       "Qwen/Qwen2-0.5B-Instruct",
}

for label, repo in MODELS.items():
    tok = AutoTokenizer.from_pretrained(repo)
    model = AutoModelForCausalLM.from_pretrained(
        repo,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    inputs = tok(PROMPT, return_tensors="pt").input_ids.to(model.device)
    print(f"\n=== {label} ===")
    stream = TextStreamer(tok, skip_prompt=True, skip_special_tokens=True)
    with torch.no_grad():
        model.generate(
            inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            streamer=stream,
            eos_token_id=tok.eos_token_id,
        )

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


=== ⚡ tiny ===
Sekretaris Daerah Kecamatan Taman Jawa, Taman Jawa

Mengenai penegakan hukum atas permintaan tindakan hukum di hadapan penduduk masyarakat, yaitu pengadilan ngerah, tentunya ada pertanyaan yang harus dipertanyakan. Apakah pengadilan ngerah dapat mengevaluasi permintaan tindakan hukum tersebut? Apa status dan kebijaksanaan dari hukuman pengadilan ngerah?

Pada dasarnya, hukuman pengadilan ngerah adalah saran atau bantuan yang diberikan oleh pihak atas kepada seseorang untuk mengurangi atau memeriksa perbuatan seseorang. Hukuman ini biasanya ditujukan untuk membantu orang tersebut memperbaiki perilaku mereka atau memberikan kesempatan untuk melanjutkan pekerjaannya.

Secara umum, hukuman pengadilan ngerah sering digunakan dalam kasus-kasus yang berdampak negatif terhadap kesejahteraan manusia. Misalnya, jika seseorang melakukan kejahatan seperti penculikan, hukuman pengadilan ngerah bisa dianggap sebagai cara yang tepat untuk menggantikan keputusan hukum dengan hukuman.

